### 0. Herramientas para generacion y medicion de datos

In [81]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import timeit
import time
from timeit import Timer
import numpy as np
import random
import string
from datetime import datetime
random.seed(42)

### 1. Merge Sort recursivo

In [23]:
def merge_sort(lista):
    tam = len(lista)
    
    if(tam==1):
        return lista
    
    mitad = tam//2
    #Para asegurarnos de que todas las particiones sean descompuestas a sus componentes individuales, 
    #llamamos recursivamente a la funcion merge sort
    izquierda = merge_sort(lista[:mitad])
    derecha = merge_sort(lista[mitad:])

    #una vez descompuesto a la unidad, se empieza a unir por las llamadas en el Call Stack
    return merge(izquierda, derecha)
    
##merge es el que realmente ordena, al ir checando los componentes        
def merge(izquierda, derecha):
    lista_ordenada =[]
    i=j=0
    
    while i < len(izquierda) and j < len(derecha):
        #compara los elementos en cada posicion de ambas listas
        if izquierda[i] < derecha[j]:
            lista_ordenada.append(izquierda[i])
            i += 1
        else:
            lista_ordenada.append(derecha[j])
            j +=1
            
    #los elemntos restantes son agregados al final de la lista
    #claro esta que solo 1 de izquierda o derecha tendra elementos que agregar
    lista_ordenada.extend(izquierda[i:])
    lista_ordenada.extend(derecha[j:])
    
    return lista_ordenada

### 2. Merge Sort Iterativo
Tambien llamado "Merge Sort de abajo hacia arriba"

In [177]:
def merge_it(lista, aux, desde, medio, hasta):
 
    k = desde #para la auxiliar
    i = desde #para la original, primera mitad
    j = medio + 1 #para la original, segunda mitad
 
    # while hasta que no halla elementos ni en la izquierda ni en la derecha
    while i <= medio and j <= hasta:
        if lista[i] < lista[j]:
            aux[k] = lista[i]
            i += 1
        else:
            aux[k] = lista[j]
            j += 1
 
        k += 1
 
    #naturalmente, se cumplira una de las 2 condiciones.
    #por lo cual debemos de copiar los elementos restantes
    #aunque es obvio que no ocupamos copiar la otra mitad, porque ya estan ordenados en su posicion
    while i < len(lista) and i <= medio:
        aux[k] = lista[i]
        k += 1
        i += 1
 
    # Copiamos a la lista original para que, ahora si, este ordenado
    for i in range(desde, hasta + 1):
        lista[i] = aux[i]

In [175]:
def mergesort_it(lista):
 
    inicio = 0
    fin = len(lista) - 1
 
    # Ordena la lista usando una lista auxiliar
    aux = lista.copy()
 
    # divide la lista en bloques de tamaño 'm'
    # m = [1, 2, 4, 8, 16…]
 
    m = 1
    while m <= (fin - inicio): #ie, mientras podamos seguir haciendo subdivisiones
 
        # m = 1, i = [0, 2, 4, 6, 8…]
        # m = 2, i = [0, 4, 8, 12…]
        # m = 4, i = [0, 8, 16…]
        # …
 
        for i in range(inicio, fin, 2*m):
            desde = i
            medio = (i + m) - 1
            hasta = min( (i + 2*m - 1), fin) 
            #llegara el punto donde i+2*m superara a fin, en ese caso no podemos llamar a posiciones que no existen
            merge_it(lista, aux, desde, medio, hasta)
 
        m = 2*m

### 3. Quick Sort

In [30]:
def particion(lista,inicio,fin):
    i = (inicio - 1)
    #inicializamos el indice del elemento mas pequeño, mientras que pivote sera el ultimo elemento
    #se que en clase lo hicimos con el primer elemento, pero el libro me enamoro. Lo siento, profesor.
    pivot = lista[fin]
    #haremos un ciclo iterando desde el elemento mas
    for j in range(inicio, fin):
        #si el valor que actualmente estamos recorriendo es menor al pivote, aumentamos el indice del elemento menor
        if lista[j] <= pivot:
            i = i + 1
            lista[i], lista[j] =  lista[j], lista[i]
        #de otro modo (si el valor actual es mayor al pivote), intercambiamos los valores
    lista[i+1],lista[fin] = lista[fin],lista[i+1]
    return (i+1)

def quickSort(lista, inicio, fin):
    #vamos a 
    if inicio < fin:
        indice_de_particion = particion(lista,inicio,fin)
        #sorteamos los elementos antes y despues de la particion
        quickSort(lista, inicio, indice_de_particion - 1)
        quickSort(lista, indice_de_particion + 1, fin)
        
def quick_sort(lista): #sobrecarga no existe en python
    quickSort(lista, 0, len(lista)-1)

### 4. Pruebas de Eficiencia

### 4.1 Generacion de Datos
Seran todas listas de 1000 numeros enteros, para evaluar de manera propia y evitar problemas externos a los procesos de Ordenamiento

Datos Ordenados (ascendente, descendiente y todos iguales)

In [234]:
arreglo_ascendiente = []
arreglo_descendiente = []
arreglo_espejo = []


def generacion_ordenados(n):
    arreglo_ascendiente.clear() 
    arreglo_descendiente.clear()
    arreglo_espejo.clear()
    for i in range(n):
        arreglo_ascendiente.append(i)
    for i in range(n):
        arreglo_descendiente.append(n-i)
    for i in range(n):
        arreglo_espejo.append(0)

Datos Aleatorios

In [233]:
arreglo_aleatorio_1 = []
arreglo_aleatorio_2 = []
arreglo_aleatorio_3 = []

def generacion_aleatorios(n):
    arreglo_aleatorio_1.clear()
    arreglo_aleatorio_2.clear()
    arreglo_aleatorio_3.clear()
    for i in range(n):
        arreglo_aleatorio_1.append(random.randint(0,i))
    for i in range(n):
        arreglo_aleatorio_2.append(random.randint(0,n))
    for i in range(n):
        arreglo_aleatorio_3.append(random.randint(-999999,999999))

#### El peor caso de Merge Sort: 
Es decir, el arreglo que genere el mayor numero de comparaciones. Para esto, tomaremos un arreglo ordenado, lo dividiremos en 2 sublistas, y alternaremos los elementos, como si estuvieramos barajeando las 2 sublistas.

In [63]:
def unir(listaPrincipal, listaIzquierda, listaDerecha, tam_izq, medio, tam_der):
    i = j =0
    while i<= medio - tam_izq:
        listaPrincipal[i]=listaIzquierda[i]
        i+=1
    while j<= tam_der - medio:
        listaPrincipal[i+j] = listaDerecha[j]
        j+=1
        
def split(listaPrincipal, listaIzquierda, listaDerecha, tam_izq, medio, tam_der):
    #este metodo separara los elementos de la lista principal de manera alternante
    #la lista izquierda tendra los elementos en la posicion par; la derecha, los de posicion impar
    i=0
    while i<= medio - tam_izq:
        listaIzquierda [i] = listaPrincipal[i*2]
        i+=1
    i=0
    while i< tam_der - medio:
        listaDerecha[i] = listaPrincipal[i*2 + 1]
        i+=1
def generarPeorCasoPrivado(listaPrincipal, tam_izq, tam_der):
    if (tam_izq<tam_der):
        medio = tam_izq + (tam_der - tam_izq)/2
        #creando las listas auxiliares
        listaIzquierda = [None] * len(listaPrincipal)
        listaDerecha = [None] * len(listaPrincipal)
        split(listaPrincipal, listaIzquierda, listaDerecha, tam_izq, medio, tam_der)
        
        #de manera recursiva, iremos "barajeando" los elemntos
        
        generarPeorCasoPrivado(listaIzquierda, tam_izq, medio)
        generarPeorCasoPrivado(listaDerecha, medio+1, tam_der)
        
        unir(listaPrincipal, listaIzquierda, listaDerecha, tam_izq, medio, tam_der)

def generarPeorCaso(lista):
    generarPeorCasoPrivado(lista, 0, len(lista)-1)

In [238]:
arreglo_barajeado=[]

def generacion_peor_caso_MS(n):
    generacion_ordenados(n)
    arreglo_barajeado = arreglo_ascendiente.copy()
    generarPeorCaso(arreglo_barajeado)

In [236]:
def inicializar(n):
    generacion_ordenados(n)
    generacion_aleatorios(n)
    generacion_peor_caso_MS(n)

### 4.2 Medicion y Comparacion

Probando El Merge Sort

In [239]:
inicializar(1000)

In [152]:
start_time = time.time()
merge_sort(arreglo_descendiente)
end_time = time.time()
t_ms_1 = (end_time - start_time)*1000
print("La prueba con el arreglo ordenado al revez %s milisegundos" % (t_ms_1))

La prueba con el arreglo ordenado al revez 4.999399185180664 milisegundos


In [162]:
start_time = time.time()
merge_sort(arreglo_espejo)
end_time = time.time()
t_ms_2 = (end_time - start_time)*1000
print("La prueba con el arreglo de 0's duro %s milisegundos" % (t_ms_2))

La prueba con el arreglo de 0's duro 5.955934524536133 milisegundos


In [154]:
start_time = time.time()
merge_sort(arreglo_ascendiente)
end_time = time.time()
t_ms_3 = (end_time - start_time)*1000
print("La prueba con el arreglo ordenado duro %s milisegundos" % (t_ms_3))

La prueba con el arreglo ordenado duro 4.968404769897461 milisegundos


In [155]:
start_time = time.time()
merge_sort(arreglo_aleatorio_3)
end_time = time.time()
t_ms_4 = (end_time - start_time)*1000
print("La prueba con el arreglo aleatorio duro %s milisegundos" % (t_ms_4))

La prueba con el arreglo aleatorio duro 7.988691329956055 milisegundos


In [168]:
start_time = time.time()
merge_sort(arreglo_barajeado)
end_time = time.time()
t_ms_5 = (end_time - start_time)*1000
print("La prueba con el arreglo barajeado duro %s milisegundos" % (t_ms_5))

RecursionError: maximum recursion depth exceeded while calling a Python object

Probando el Merge Sort Iterativo

In [213]:
inicializar()

In [178]:
start_time = time.time()
mergesort_it(arreglo_descendiente)
end_time = time.time()
t_ms_1 = (end_time - start_time)*1000
print("La prueba con el arreglo ordenado al revez %s milisegundos" % (t_ms_1))

La prueba con el arreglo ordenado al revez 11.020183563232422 milisegundos


In [179]:
start_time = time.time()
mergesort_it(arreglo_espejo)
end_time = time.time()
t_ms_2 = (end_time - start_time)*1000
print("La prueba con el arreglo de 0's duro %s milisegundos" % (t_ms_2))

La prueba con el arreglo de 0's duro 8.01396369934082 milisegundos


In [180]:
start_time = time.time()
mergesort_it(arreglo_ascendiente)
end_time = time.time()
t_ms_3 = (end_time - start_time)*1000
print("La prueba con el arreglo ordenado duro %s milisegundos" % (t_ms_3))

La prueba con el arreglo ordenado duro 5.991697311401367 milisegundos


In [188]:
start_time = time.time()
mergesort_it(arreglo_aleatorio_3)
end_time = time.time()
t_ms_4 = (end_time - start_time)*1000
print("La prueba con el arreglo aleatorio duro %s milisegundos" % (t_ms_4))

La prueba con el arreglo aleatorio duro 7.984638214111328 milisegundos


In [231]:
inicializar()
generarPeorCaso(arreglo_ascendiente)
arreglo_barajeado = arreglo_ascendiente.copy()

In [232]:
start_time = time.time()
mergesort_it(arreglo_barajeado)
end_time = time.time()
t_ms_5 = (end_time - start_time)*1000
print("La prueba con el arreglo barajeado duro %s milisegundos" % (t_ms_5))

La prueba con el arreglo barajeado duro 17.08054542541504 milisegundos


Probando el QuickSort

In [163]:
inicializar()

In [164]:
start_time = time.time()
quick_sort(arreglo_descendiente)
end_time = time.time()
t_ms_1 = (end_time - start_time)*1000
print("La prueba con el arreglo ordenado al revez %s milisegundos" % (t_ms_1))

La prueba con el arreglo ordenado al revez 149.02257919311523 milisegundos


In [165]:
start_time = time.time()
quick_sort(arreglo_espejo)
end_time = time.time()
t_ms_2 = (end_time - start_time)*1000
print("La prueba con el arreglo de 0's duro %s milisegundos" % (t_ms_2))

La prueba con el arreglo de 0's duro 207.0600986480713 milisegundos


In [166]:
start_time = time.time()
quick_sort(arreglo_ascendiente)
end_time = time.time()
t_ms_3 = (end_time - start_time)*1000
print("La prueba con el arreglo ordenado duro %s milisegundos" % (t_ms_3))

La prueba con el arreglo ordenado duro 209.13124084472656 milisegundos


In [167]:
start_time = time.time()
quick_sort(arreglo_aleatorio_3)
end_time = time.time()
t_ms_4 = (end_time - start_time)*1000
print("La prueba con el arreglo aleatorio duro %s milisegundos" % (t_ms_4))

La prueba con el arreglo aleatorio duro 5.986690521240234 milisegundos


In [169]:
start_time = time.time()
quick_sort(arreglo_barajeado)
end_time = time.time()
t_ms_5 = (end_time - start_time)*1000
print("La prueba con el arreglo barajeado duro %s milisegundos" % (t_ms_5))

La prueba con el arreglo barajeado duro 0.9019374847412109 milisegundos


Probando Mismo Ordenamiento, diferente cantidad de datos

In [249]:
inicializar(500)#quinientos
start_time = time.time()
quick_sort(arreglo_espejo)
end_time = time.time()
t_qs_1 = (end_time - start_time)*1000
print("La prueba con el arreglo de 2000 0's duro %s milisegundos" % (t_qs_1))

La prueba con el arreglo de 500 0's duro 45.99285125732422 milisegundos


In [252]:
inicializar(2000) # Dos Mil
start_time = time.time()
quick_sort(arreglo_espejo)
end_time = time.time()
t_qs_2 = (end_time - start_time)*1000
print("La prueba con el arreglo de 2000 0's duro %s milisegundos" % (t_qs_2))

La prueba con el arreglo de 2000 0's duro 740.1235103607178 milisegundos


In [253]:
inicializar(10000) #diez Mil

start_time = time.time()
quick_sort(arreglo_espejo)
end_time = time.time()
t_qs_3 = (end_time - start_time)*1000
print("La prueba con el arreglo de 10,000 0's duro %s milisegundos" % (t_qs_3))

RecursionError: maximum recursion depth exceeded in comparison

In [258]:
generacion_aleatorios(500)
start_time = time.time()
merge_sort(arreglo_aleatorio_2)
end_time = time.time()
t_ms_1 = (end_time - start_time)*1000
print("La prueba con el arreglo aleatorio de 500 tardo %s milisegundos" % (t_ms_1))

La prueba con el arreglo aleatorio de 500 tardo 6.010532379150391 milisegundos


In [259]:
generacion_aleatorios(2000)
start_time = time.time()
merge_sort(arreglo_aleatorio_2)
end_time = time.time()
t_ms_1 = (end_time - start_time)*1000
print("La prueba con el arreglo aleatorio de 2000 tardo %s milisegundos" % (t_ms_1))

La prueba con el arreglo aleatorio de 2000 tardo 21.14558219909668 milisegundos


In [260]:
generacion_aleatorios(10000)
start_time = time.time()
merge_sort(arreglo_aleatorio_2)
end_time = time.time()
t_ms_1 = (end_time - start_time)*1000
print("La prueba con el arreglo aleatorio de 10,000 tardo %s milisegundos" % (t_ms_1))

La prueba con el arreglo aleatorio de 10,000 tardo 96.16494178771973 milisegundos


In [261]:
generacion_aleatorios(100000)
start_time = time.time()
merge_sort(arreglo_aleatorio_2)
end_time = time.time()
t_ms_1 = (end_time - start_time)*1000
print("La prueba con el arreglo aletorio de 100,000 tardo %s milisegundos" % (t_ms_1))

La prueba con el arreglo aletorio de 100,000 tardo 1424.0870475769043 milisegundos
